<a href="https://colab.research.google.com/github/magicznylukasz/uczenie_maszynowe_w_finansach/blob/main/case3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance backtesting scikit-learn ta

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import SignalStrategy, crossover
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from ta.volatility import BollingerBands

In [ ]:
data = yf.download('MSFT', start='2024-01-01', end='2024-05-06')
data['Return'] = data['Close'].pct_change()
data.dropna(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
data['SMA10'] = data['Close'].rolling(window=10).mean()
data['SMA20'] = data['Close'].rolling(window=20).mean()
data['RSI'] = RSIIndicator(data['Close']).rsi()
data['MACD'] = MACD(data['Close']).macd()
stoch = StochasticOscillator(data['High'], data['Low'], data['Close'])
data['Stoch'] = stoch.stoch()
bb = BollingerBands(data['Close'])
data['BB_high'] = bb.bollinger_hband()
data['BB_low'] = bb.bollinger_lband()

In [ ]:
features = ['SMA10', 'SMA20', 'RSI', 'MACD', 'Stoch', 'BB_high', 'BB_low', 'Volume']
data = data.dropna()
X = data[features]
y = np.where(data['Return'].shift(-1) > 0, 1, 0)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Define parameter grid for GridSearch
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 10]
}

In [ ]:
# Train the model using GridSearchCV
model = GradientBoostingClassifier(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Best model from GridSearch
best_model = grid_search.best_estimator_

# Predict signals using the best model
data['Signal'] = best_model.predict(X)
data['Position'] = data['Signal'].shift()

# Define the trading strategy
class MLStrategy(SignalStrategy):
    def init(self):
        super().init()
        self.set_signal(data['Position'].to_numpy())

# Run backtest
bt = Backtest(data, MLStrategy, cash=10000, commission=.002, exclusive_orders=True)
output = bt.run()
bt.plot()

# Print backtest results and model accuracy
print(output)
print("Accuracy of the model on test set:", accuracy_score(y_test, best_model.predict(X_test)))

<ipython-input-16-2d32956a7f93>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'] = best_model.predict(X)
<ipython-input-16-2d32956a7f93>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Position'] = data['Signal'].shift()
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-pa

Start                     2024-02-08 00:00:00
End                       2024-05-03 00:00:00
Duration                     85 days 00:00:00
Exposure Time [%]                   91.666667
Equity Final [$]                  9976.730624
Equity Peak [$]                   10012.08564
Return [%]                          -0.232694
Buy & Hold Return [%]               -1.799035
Return (Ann.) [%]                   -0.973681
Volatility (Ann.) [%]                0.809601
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.458645
Avg. Drawdown [%]                   -0.162999
Max. Drawdown Duration       43 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                   21
Win Rate [%]                        52.380952
Best Trade [%]                       2.280979
Worst Trade [%]                     -3.969176
Avg. Trade [%]                    